### RAG 데모 시나리오(테이블 데이터)
- 이 시나리오는 테이블 컬럼 정보를 임베딩하여 RAG 실행하는 시나리오입니다.

### 0. 파이썬 라이브러리 로딩 및 오라클 DB 접속

In [ ]:
# 파이썬 라이브러리 로딩 및 오라클 DB 접속
import oracledb,oml
import re, os, time
import pandas as pd
import import_ipynb
from IPython.display import display
from user_functions import get_conn_cursor,get_db_version,exec_sql,exec_sql_rows,dml_sql,ddl_sql,clean_text

# DB 접속
conn, cursor = get_conn_cursor("vector","vector","localhost:1521/freepdb1")

# DB 버전 확인
get_db_version(cursor)

### 1. 테이블 데이터 확인
- 벡터 컬럼 초기화 및 인덱스 삭제
- broad_tbl 테이블 데이터 조회

In [ ]:
# 벡터 컬럼 초기화 및 데이터 조회

update_q1 = """update broad_tbl set embed_vector = null"""
dml_sql(conn,cursor,update_q1)

print("벡터 컬럼 초기화 완료")

# 벡터 인덱스 삭제
drIdx = """drop index if exists broad_tbl_v_idx """
ddl_sql(cursor,drIdx)
print("벡터 인덱스 삭제")

# 초기 데이터 조회
import pandas as pd
data_q = """ select * from broad_tbl order by 2,1"""
exec_sql(cursor,data_q)

### 2. "TITLE"(보도제목) 값 임베딩
- 보도제목을 임베딩하고 인데싱까지 실행
- 임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2

In [ ]:
# "TITLE"(문서제목) 값 임베딩 
# 임베딩 모델 : PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2
# 임베딩 유틸리티 : utl_to_embedding

embedding_param = """{"provider":"database", "model":"PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2"}"""
# execute embedding using update query
data_q = """
SELECT  보도번호, 보도제목 FROM broad_tbl
"""
# update query
embed_q = """update broad_tbl set embed_vector = 
                               (dbms_vector.utl_to_embedding(:title, json(:embedding_param)))
             where 보도번호 = :dno"""

binds = []

import time
start_time = time.time()

cursor.execute(data_q)
while True:
    row = cursor.fetchone()
    if row is None:
        break
    dno = row[0]
    title = row[1]
    embedding_param = embedding_param
    binds.append([title,embedding_param,dno])
#print(binds)
print("모델 PARAPHRASE_MULTILINGUAL_MPNET_BASE_V2 with oracle 이용하여 임베딩 생성중입니다......")
cursor.executemany(embed_q,binds)
conn.commit()

execute_time = time.time() - start_time
print("임베딩 시간(Ollama 임베딩 + 테이블 저장) :", execute_time)
print("임베딩 완료")

# 벡터 인덱싱
crIdx = """create vector index broad_tbl_v_idx on broad_tbl(embed_vector) ORGANIZATION INMEMORY NEIGHBOR GRAPH WITH DISTANCE COSINE """
ddl_sql(cursor,crIdx)
print("벡터 인덱싱 완료")

### 3. RAG 실행
- RAG 실행 순서는 1. 사용자 질의 문장에 대한 DB 벡터 검색, 2. 벡터 검색 결과를 참조하는 프롬프트 생성, 3. 프로프트를 LLM에 전달, 답변 요청, 4. LLM 답변
- langchain API 사용
- LLM : LLAMA3.2 on local ollama

In [ ]:
# RAG 실행
## LLM : llama3.2 on ollama

# 1. 질의 입력 및 벡터 검색
print("질문 입력 예: 국방부에서 보도한 2025년도 군무원 채용 필기시험 보도자료 찾아줘 ")
user_q = input("찾으실 문장을 입력하세요:  ")

top_k = 3

# 벡터 쿼리
content_q = """ 
select dt.보도번호, dt.보도제목, dt.보도일자, dt.부처명 as 주무부처, dt.주무부서, round(1- vector_distance(embed_vector, 
                                             (dbms_vector.utl_to_embedding(:user_q, json(:embedding_param))), 
                                          cosine),2) 유사도
from broad_tbl dt
order by vector_distance(embed_vector,
                        (dbms_vector.utl_to_embedding(:user_q, 
                                                     json(:embedding_param))), cosine)
fetch approximate first :top_k rows only
"""
import time
start_time = time.time()
print("벡터검색중입니다......")
rows = exec_sql(cursor,content_q,{"user_q":user_q,"embedding_param":embedding_param,"top_k":top_k})
execute_time = time.time() - start_time
print(" 벡터검색 시간(질문 임베딩 + 벡터쿼리 실행):",execute_time)

# convert to content
content = rows
# 3. 프로프트 생성

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

user_question = user_q

## 프롬프트 생성 
template = """
CONTEXT 정보에서만 답변을 작성해주세요.
답변은 보도제목, 보도번호, 주무부처, 주무부서, 작성일자를 포함하세요. 답변출력에는 유사도가 가장 높은 것 하나를 출력하세요.
CONTEXT : {context}
[INST]QUESTION : {question}[/INST]
"""
print("-----------------------------------------------------------")
print("백터검색 결과를 이용하여 langchain-ollama 프롬프트를 생성했습니다.")
print("프롬프트\n",template)
## 프롬프트 생성

prompt = ChatPromptTemplate.from_template(template)
#print("생성된 프롬프트")
#display(prompt)

# LLM 답변 생성

print("llama3.2가 답변을 생성중입니다......")

model = OllamaLLM(model="llama3.2", max_new_tokens= 2048 ,temperature=0.1, base_url='http://180.68.194.221:11434')
chain = prompt | model 

# LLM 응답
result = chain.invoke({"context":content,"question": user_question})
print("--------------------------------------------------------------")
print("LLM 응답")
print(result)

-------------------------

시나리오 종료